In [ ]:
!pip uninstall -y tensorflow
!pip install tensorflow-gpu==1.14.0
!pip install keras==2.2.5
!pip install 'h5py==2.10.0'

Found existing installation: tensorflow 2.8.0
Uninstalling tensorflow-2.8.0:
  Successfully uninstalled tensorflow-2.8.0
     |████████████████████████████████| 377.1 MB 8.8 kB/s 
     |████████████████████████████████| 488 kB 91.6 MB/s 
     |████████████████████████████████| 50 kB 6.7 MB/s 
     |████████████████████████████████| 3.1 MB 68.3 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
     |████████████████████████████████| 336 kB 7.2 MB/s 
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
     |████████████████████████████████| 2.9 MB 8.6 MB/s 
  Atte

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/daydayup/Morpho')	# 设置工作路径

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12394369102423128752
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 13236313232956875485
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 11171241377108803926
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15964005991
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16451096445254548253
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


In [ ]:
import keras
import os
from keras import layers,models,optimizers
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten
from keras.layers import *   
from keras.models import Sequential, Model
import keras.backend as K
from keras import regularizers
from keras.applications.xception import Xception,preprocess_input
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
import matplotlib.pylab as plt
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import random
import sys
import numpy as np
from keras import backend as K
from keras.models import load_model
from keras.optimizers import SGD
import tensorflow as tf
from tensorflow import keras as ke
from keras.preprocessing.image import img_to_array, ImageDataGenerator#图片转为array
from keras.utils import to_categorical#相当于one-hot
from imutils import paths
import cv2
import numpy as np
import random
import os
from keras.callbacks import TensorBoard, ModelCheckpoint
import numpy as np

from keras.layers import Conv2D, Dense, LeakyReLU, Dropout, Input
from keras.layers import Reshape, Conv2DTranspose, Flatten
from keras.layers import BatchNormalization
from keras.models import Model
from keras import optimizers
import keras
from decimal import Decimal, getcontext
from PIL import Image
from tensorflow.contrib.data import prefetch_to_device, shuffle_and_repeat, map_and_batch
from tensorflow.contrib.opt import MovingAverageOptimizer
import argparse
import tensorflow as tf
import scipy.misc
import numpy as np
import os
from glob import glob

import tensorflow as tf
import tensorflow.contrib.slim as slim
from keras.datasets import cifar10, mnist



import os
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# 使用Gan训练cifra10，测试使用，由于BigGan遇到了处理不了的问题，回来试一下Gan

In [ ]:
# conf
latent_dim = 28
height = 28
width = 28
channels = 1

generator_input = keras.Input(shape=(latent_dim,))

# Generate part
x = layers.Dense(128 * 14 * 14)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((14, 14, 128))(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
# use tanh as the activation function in Generator part
x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)
generator = keras.models.Model(generator_input, x)
generator.summary()

In [ ]:
discriminator_input = layers.Input(shape=(height, width, channels))
x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)

# Dropout for preventing Discriminator part controlling the whole network!!!!  
x = layers.Dropout(0.8)(x)

# Classification
x = layers.Dense(1, activation='sigmoid')(x)

discriminator = keras.models.Model(discriminator_input, x)
discriminator.summary()

# set optimizer -> learning rate decay and gradient clipping
discriminator_optimizer = keras.optimizers.RMSprop(lr=0.00001, clipvalue=1.0, decay=1e-8)
discriminator.compile(optimizer=discriminator_optimizer, loss='binary_crossentropy')

In [ ]:
# set weights of Discriminator part to non-trainable for Generator part
discriminator.trainable = False

gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)

gan_optimizer = keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')

In [ ]:
# set steps and parameters
iterations = 4500
batch_size = 20

# start training
start = 0
for step in range(iterations):
    
    # train Discriminator part
    # initial input data/latent vectors
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
    # generate faked pic
    generated_images = generator.predict(random_latent_vectors)
    # mix faked images with real images
    stop = start + batch_size
    real_images = train_data[start: stop]
    combined_images = np.concatenate([generated_images, real_images])
    labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])
    # mix noise
    labels += 0.05 * np.random.random(labels.shape)
    # train the Discriminator part
    d_loss = discriminator.train_on_batch(combined_images, labels)

    # train GAN part
    # initial input data/latent vectors
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
    # set labels for faked images
    misleading_targets = np.zeros((batch_size, 1))
    # train GAN part and freeze discriminator
    a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)

    # judge when to stop
    start += batch_size
    if start > len(train_data) - batch_size:
    start = 0

    # output log
    if step % 100 == 0:
        print('discriminator loss at step %s: %s' % (step, d_loss))
        print('adversarial loss at step %s: %s' % (step, a_loss))

In [ ]:
# generate test data(#40)
random_latent_vectors = np.random.normal(size=(40, latent_dim))
# decode input data to faked images
generated_images = generator.predict(random_latent_vectors)

# print the shape of faked images
print(np.array(generated_images[2]).shape)

# plot
for i in range(generated_images.shape[0]):
    img = image.array_to_img(generated_images[i] * 255., scale=False)
    plt.figure()
    plt.imshow(img,cmap='gray')
    
plt.show()

# 关于sneaker数据集的各种功能函数，基于keras，绝大部分作废

In [ ]:
seed = 42

os.environ['PYTHONHASHSEED']=str(seed)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
os.environ['HOROVOD_FUSION_THRESHOLD']='0'
random.seed(seed)
np.random.seed(seed)
tf.random.set_random_seed(seed)
tf.set_random_seed(seed)

In [ ]:
# conf
channel = 3
height = 128 #300
width = 128 #400
class_num = 2 # 4
#norm_size = 32#参数
batch_size = 64
epochs = 200


train_dir = './drive/MyDrive/daydayup/dataset/sneaker_nonsneaker/sneaker_nonsneaker/training' # ../data/dataset/train
validation_dir = './drive/MyDrive/daydayup/dataset/sneaker_nonsneaker/sneaker_nonsneaker/testing' # ../data/dataset/val
# train_dir = './drive/MyDrive/daydayup/dataset/filteredDataset/training'
# validation_dir = './drive/MyDrive/daydayup/dataset/filteredDataset/testing'
save_tl_dir = "./drive/MyDrive/daydayup/Morpho/predict/TLCheckpoint"
save_ft_dir = "./drive/MyDrive/daydayup/Morpho/predict/FTCheckpoint"
save_Direct_dir = "./drive/MyDrive/daydayup/Morpho/predict/DirectCheckpoint"


totalTrain = len(list(paths.list_images(train_dir)))
totalVal = len(list(paths.list_images(validation_dir)))
print(totalTrain)
print(totalVal)

1787
440


In [ ]:
source_train_dir_positive = os.path.join(train_dir, 'positive')
source_train_dir_negative = os.path.join(train_dir, 'negative')
source_validation_dir_positive = os.path.join(validation_dir, 'positive')
source_validation_dir_negative = os.path.join(validation_dir, 'negative')

In [ ]:
import os
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:



# train_dir_positive = os.path.join(train_dir, '/1')
# train_dir_negative = os.path.join(train_dir, '/0')
# validation_dir_positive = os.path.join(validation_dir, '/1')
# validation_dir_negative = os.path.join(validation_dir, '/0')

def pilConvertJPG(path):
    for a, _, c in os.walk(path):
        for n in c:
          # print(n)
          if '.jpg' in n or '.png' in n or '.jpeg' in n or '.JPEG' in n:
              img = Image.open(os.path.join(a, n))
              rgb_im = img.convert('RGB')
              error_img_path = os.path.join(a,n)
              os.remove(error_img_path)
              n = ''.join(filter(lambda n: ord(n) < 256, n))
              jpg_img_path = os.path.splitext(os.path.join(a, n).replace('\\', '/'))[0]
              jpg_img_path += '.jpg'
              # print(jpg_img_path)
              rgb_im.save(jpg_img_path)
          else:
            print("error:", n)




In [ ]:
pilConvertJPG("./drive/MyDrive/daydayup/Morpho/BigGan/dataset/groundTruth")

In [ ]:
pilConvertJPG(source_train_dir_positive)
pilConvertJPG(source_train_dir_negative)
pilConvertJPG(source_validation_dir_positive)
pilConvertJPG(source_validation_dir_negative)

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [ ]:
def dataprocess(train_dir, validation_dir,height, width, batch_size):
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')
    test_datagen = ImageDataGenerator(rescale=1./255)
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(height, width),
        batch_size= batch_size,
        class_mode='categorical')
    validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(height, width),
        batch_size= batch_size,
        class_mode='categorical')
    return train_generator, validation_generator




In [ ]:
train_generator, validation_generator = dataprocess(train_dir, validation_dir, height, width, batch_size)

Found 1787 images belonging to 2 classes.
Found 440 images belonging to 2 classes.


In [ ]:
# generator使用方法，先利用生成器生成数据然后训练
for i, j in train_generator:
  print(i.shape)


(64, 112, 112, 3)


KeyboardInterrupt: ignored

In [ ]:
# 编写网络

class Generator:
  def neural(latent_dim):
    input_shape = (latent_dim,)

    inputs = Input(shape= input_shape)

    

    # conv_base = VGG16(include_top=False, weights='imagenet', input_shape=input_shape)
    # x = conv_base.output

    # UpSample = MaxPooling2D(pool_size=(9, 9), strides=(1, 1),padding = 'same', name='MaxPooling2D')(inputs)
    # UpSample = Dropout(0.5)(UpSample)
    # UpSample = Conv2D(256,(1,1))(UpSample)
    # UpSample = BatchNormalization()(UpSample)
    # UpSample = Activation('relu')(UpSample)
    # UpSample = Dropout(0.5)(UpSample)
    # UpSample = Conv2D(64,(1,1))(UpSample)
    # UpSample = BatchNormalization()(UpSample)
    # UpSample = Activation('relu')(UpSample)
    # UpSample = Dropout(0.5)(UpSample)
    # UpSample = Flatten(name='flatten')(UpSample)
    # UpSample = Dense(classes)(UpSample)
    # UpSample = BatchNormalization()(UpSample)
    # predictions = Activation('softmax')(UpSample)

    model = Model(inputs=inputs, outputs=predictions)
    

    return model

generator_input = keras.Input(shape=(latent_dim,))

# Generate part
x = layers.Dense(128 * 14 * 14)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((14, 14, 128))(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
# use tanh as the activation function in Generator part
x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)
generator = keras.models.Model(generator_input, x)
generator.summary()

In [ ]:
## 废案,想自己写generate函数来生成图片，但是遇见了无法处理的问题，主要还是读取那一块的问题，tf自带的读取方法没有理解，不过好在keras的读取方式也可以用


# for i in os.listdir(train_dir):
#   print(i)


# 这个函数用于返回符合,可以使用正则路径，*表示任意字符
# path_list = tf.data.Dataset.list_files(train_path + "*.jpg")




# 定义一个读取图片的函数
def read_image(dirPath, batchSize, k, classNum = 2):
    '''
    :dirPath: 数据集读取路径
    :batchSize: 获得的数据数量
    :k: 记录历史获取次数
    :yield: 该步图片张量列表与图片标签列表
    '''
    historyCheck = np.zeros(classNum)
    ratioNum = []
    splitRatio = np.random.dirichlet(np.ones(classNum),size=1)
    for i in range(classNum):
      ratioNum.append(int(splitRatio[0][i] * batchSize))

    for className in os.listdir(dirPath):
      content = os.path.join(dirPath, str(className))
      data = []  # 图片聊表
      labels = []  # 图片标签列表

      path_list = tf.data.Dataset.list_files(content + "*.jpg")

      # 根据文件路径列表依次读取
      for i in path_list:
          image_temp = tf.io.read_file(i)  # tesnsorflow的io读取文件
          image_temp = tf.image.decode_jpeg(image_temp)  # 根据图片的格式进行编码转化为张量，这里图片是jpg格式
          data.append(image_temp)  # 图片加入到数据集
          labels.append(str(className))  # 获取文件名加入到标签，这里要张量i转化为字符串

    

    for index, item in enumerate(ratioNum):
      historyCheck[index] = historyCheck[index] + item
    yield np.array(data), np.array(labels)


# 读取训练图片
train_images, train_labels = read_image(train_dir)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
